In [1]:
import os
import openai
import sys

openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
llm_name = "gpt-3.5-turbo"

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

persist_directory = "docs/chroma/"
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

/Users/mag7273/mambaforge/envs/llm-cv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [13]:
question = "What are Mauricio's skills?"
docs = vectordb.search(question, k=3, search_type="mmr", fetch_k=6)
docs

[Document(page_content='Mauricio Tec\nGoogle Scholar ὑ7mauriciogtec.com Boston, MA\nAbout Me\nMy current work seeks to advance the applicability of reinforcement learning in real-world settings, often integrating\ntools from Bayesian inference, causality, and deep learning.\n\nI am applying these methods at Harvard University to\nimprove climate disaster alerting systems that make decisions based on temporal and local data.', metadata={'page': 0, 'source': 'https://mauriciogtec.com/_static/cv.pdf'}),
 Document(page_content='Skills\n•Programming Languages : Python (preferred); Julia, R (proficient); C++ (intermediate);\n•High-performance Computing : Slurm (advanced); AWS/Azure Cloud (intermediate);\n•Data Science : SQL (advanced); tidyverse, ggplot, pandas, ggplot, networkx (proficient); NLP (advanced);\n•Development and Pipelines : Git, Docker, SnakeFlow, Linux (advanced);\n•Deep Learning : PyTorch, Tensorflow, Image segmentation/Object detection, Spatial methods, (proficient); GNNs,\n

In [8]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

/Users/mag7273/mambaforge/envs/llm-cv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


### Retrieval QA

In [9]:
from langchain.chains import RetrievalQA

In [14]:
qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vectordb.as_retriever(search_type="mmr", fetch_k=6, k=3)
)

In [15]:
result = qa_chain({"query": question})
result

/Users/mag7273/mambaforge/envs/llm-cv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': "What are Mauricio's skills?",
 'result': "Mauricio's skills include programming languages such as Python, Julia, R, and C++, high-performance computing using Slurm and AWS/Azure Cloud, data science with SQL, tidyverse, ggplot, pandas, ggplot, and networkx, NLP, development and pipelines using Git, Docker, SnakeFlow, and Linux, deep learning with PyTorch, Tensorflow, image segmentation/object detection, spatial methods, GNNs, attention-based models, multi-GPU, and distributed training, experiment design/sequential decision-making including active learning, reinforcement learning, and tree search, and statistics including Bayesian inference, causal inference, and A/B testing."}

### Prompt

In [31]:
from langchain.prompts import PromptTemplate

template = """
Intructions:

You are a chatbot named 'Mauricio Tec's Live CV' designed to provide specific information about Mauricio's professional and academic background. You will encounter questions about Mauricio's key projects, his work on deep learning, his most cited works, and his research on spatial causal inference. To answer these inquiries, you will reference and analyze the content of his papers and data from Google Scholar.

You must emphasize accuracy and detail in discussing his work, always maintaining a professional tone. If a query is about a topic not covered by the available material, you should politely state that the information is not within your provided resources. You're expected to guide users to understand Mauricio's research contributions and academic impact, facilitating a comprehensive insight into his scholarly achievements.

If someone asks about you, respond in first person as if you were Mauricio. If they ask about Mauricio, respond in the third person about him. You should always verify the information in cv.pdf as the main source.

If a questions is not about Mauricio (or you), refuse to answer and politely say that "You are an application with exclusive purpose of being a live CV for Mauricio"

Below are links to Mauricio's paper, that you may use for retrieval:
- Covid-19 model, published at PNAS: https://www.pnas.org/doi/full/10.1073/pnas.2113561119
- The Spatial Confonfounding Environment, published at CLeAR: https://www.cclear.cc/2023/AcceptedDatasets/tec23a.pdf
- Adversarial Intrinsic Motivation, published at NeurIPS: https://proceedings.neurips.cc/paper/2021/file/486c0401c56bf7ec2daa9eba58907da9-Paper.pdf
- Bayesian Non-Parametric Adjustment for Confounding, published at Biometrics: https://arxiv.org/abs/2203.11798#:~:text=Analysis%20of%20observational%20studies%20increasingly,for%20estimation%20of%20causal%20effects.
- Weather2vec, published at AAAI:
https://ojs.aaai.org/index.php/AAAI/article/view/26696
- Towards a Real-Time, Low-Resource, End-to-end Object Detection Pipeline for Robot Soccer, published at Robocup 2022: https://www.cs.utexas.edu/~pstone/Papers/bib2html-links/RoboCup2022-nskiran.pdf
- Watch Where You’re Going! Gaze and Head Orientation as Predictors for Social Robot Navigation, published at IEEE ICRA conference:
https://ieeexplore.ieee.org/document/9561286

You may also answer questions that you can analyze from my Github repository, such as the (good) 
quality and diversity of my code: https://github.com/mauriciogtec

Below you will be given a context and a question you must answer based on the above and the context.

Context:
{context}

Answer the following question(s):
{question}

Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
QA_CHAIN_PROMPT

PromptTemplate(input_variables=['context', 'question'], template='\nIntructions:\n\nYou are a chatbot named \'Mauricio Tec\'s Live CV\' designed to provide specific information about Mauricio\'s professional and academic background. You will encounter questions about Mauricio\'s key projects, his work on deep learning, his most cited works, and his research on spatial causal inference. To answer these inquiries, you will reference and analyze the content of his papers and data from Google Scholar.\n\nYou must emphasize accuracy and detail in discussing his work, always maintaining a professional tone. If a query is about a topic not covered by the available material, you should politely state that the information is not within your provided resources. You\'re expected to guide users to understand Mauricio\'s research contributions and academic impact, facilitating a comprehensive insight into his scholarly achievements.\n\nIf someone asks about you, respond in first person as if you we

In [32]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_type="mmr", fetch_k=10, k=3),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [33]:
result = qa_chain({"query": question})

question = "What are Mauricio's skills in reinforcement learning?"
print(result["result"])

Mauricio Tec has skills in reinforcement learning, specifically in advancing its applicability in real-world settings. He integrates tools from Bayesian inference, causality, and deep learning to improve climate disaster alerting systems at Harvard University.
